<a href="https://colab.research.google.com/github/GOE84/Gold-Price-Prediction/blob/main/Gold_Price_Prediction_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gold Price Prediction

**# นำเข้าไลบรารีที่จำเป็นสำหรับการวิเคราะห์ข้อมูล**


*   `import yfinance as yf` : ดึงข้อมูลทางการเงินจาก Yahoo Finance
*   `import pandas as pd` : สำหรับการจัดการและวิเคราะห์ข้อมูลในรูปแบบตาราง `(DataFrame)`
*   `import matplotlib.pyplot as plt` : นำเข้าโมดูล pyplot จากไลบรารี matplotlib ซึ่งใช้สำหรับสร้างกราฟและภาพข้อมูล โดยตั้งชื่อย่อว่า `plt`


In [ ]:
import sys
!{sys.executable} -m pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# Retrieve gold price data.


---



`yfinance` เพื่อดาวน์โหลดข้อมูล`ราคาทองคำ` โดยใช้สัญลักษณ์ `'GC=F'` ซึ่งเป็นสัญลักษณ์สำหรับทองคำ และกำหนดช่วงเวลาเริ่มต้นที่ 1 มกราคม 2010 และสิ้นสุดที่ 31 ธันวาคม 2024 ค่ะ ข้อมูลที่ดาวน์โหลดมาจะถูกเก็บไว้ในตัวแปรชื่อ `gold_data`\

In [ ]:
gold_data = yf.download("GC=F", start="2010-01-01", end="2024-12-31")

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(gold_data.index, gold_data['Close'], label='Gold Close Price')
plt.title('Gold Price (GC=F) Over Time')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print(gold_data.head())



> เซลล์นี้จะแสดงข้อมูล 5 แถวแรกของ DataFrame ที่ชื่อว่า gold_data



In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(13, 8))


axes[0].plot(gold_data.index, gold_data['Volume'], label='Gold Trading Volume', color='orange')
axes[0].set_title('Gold Trading Volume (GC=F) Over Time')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Volume')
axes[0].legend()
axes[0].grid(True)

gold_monthly = gold_data['Close'].resample('ME').mean()

axes[1].plot(gold_monthly, label='Monthly Average Price', color='blue') # Changed color for distinction
axes[1].set_title('Monthly Average Gold Price')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Price (USD)')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Model format

**ARIMA (AutoRegressive Integrated Moving Average)**
> (Traditional): ใช้โมเดล ARIMA

**โมเดลนี้ประกอบด้วย 3 ส่วนหลักที่เราต้องหาค่าที่ดีที่สุด (p, d, q):**
*   p (AR): การเอาค่าในอดีตมาทำนายอนาคต
*   d (I): การทำให้ข้อมูล "นิ่ง" (Stationary) โดยการลบค่าปัจจุบันด้วยค่าก่อนหน้า
*   q (MA): การเอาความผิดพลาด (Error) ในอดีตมาช่วยปรับจูน



พยากรณ์ข้อมูลอนุกรมเวลา `(Time Series)`









In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller

In [ ]:
# 1. เช็คความนิ่งของข้อมูล (Stationarity Check)
def check_stationarity(series):
    result = adfuller(series)
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    if result[1] <= 0.05:
        print("ข้อมูลนิ่งแล้ว (Stationary)")
    else:
        print("ข้อมูลยังไม่นิ่ง (Non-Stationary) ต้องทำการ Diff")

check_stationarity(gold_monthly.dropna())

**สร้างและฝึกสอนโมเดล ARIMA**

> หมายเหตุ: (5,1,0) เป็นค่าเริ่มต้นทั่วไป p=5, d=1, q=0



In [ ]:
model = ARIMA(gold_monthly, order=(2, 1, 1))
model_fit = model.fit()

In [ ]:

forecast_steps = 12
forecast = model_fit.get_forecast(steps=forecast_steps)
forecast_mean = forecast.predicted_mean
conf_int = forecast.conf_int()

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(gold_monthly, label='Historical Price')
plt.plot(forecast_mean, color='red', label='Forecast (Next 12 Months)')
plt.fill_between(conf_int.index, conf_int.iloc[:, 0], conf_int.iloc[:, 1], color='pink', alpha=0.3)
plt.title('Gold Price Prediction with ARIMA')
plt.legend()
plt.show()

# "ค่าที่ดีที่สุด" (Optimal Parameters) (2,1,1)



In [ ]:
!pip install pmdarima

In [ ]:
pip install --upgrade statsmodels

In [ ]:
pip install --upgrade scipy

In [ ]:
pip install --upgrade pmdarima

In [ ]:
pip install statsmodels==0.14.0 scipy==1.10.1

In [ ]:
import pmdarima as pm

# ใช้ auto_arima เพื่อค้นหาค่า p, d, q ที่ดีที่สุด
# stepwise=True ช่วยให้หาเจอไวขึ้นโดยไม่ต้องรันทุกความเข้มข้น
auto_model = pm.auto_arima(gold_monthly,
                          seasonal=False, # เบื้องต้นลองแบบไม่คิดฤดูกาลก่อน
                          trace=True,      # แสดงค่าที่โมเดลกำลังทดสอบ
                          error_action='ignore',
                          suppress_warnings=True,
                          stepwise=True)

print(f"ค่า (p, d, q) ที่ดีที่สุดคือ: {auto_model.order}")

# Backtesting


> การเอาโมเดลไปลองทำนายในอดีต `(เช่น ทำนายปี 2024 โดยแกล้งทำเป็นว่าเราไม่รู้ราคาจริง)` แล้วเอามาเทียบกับราคาที่เกิดขึ้นจริง เพื่อดูว่าถ้าเราเชื่อ AI ตัวนี้ เราจะทายถูกหรือผิดกี่เปอร์เซ็นต์

1.   Train Set: ข้อมูลในอดีต (เช่น 2004 - 2023) ให้โมเดลเรียนรู้
2.   Test Set: ข้อมูลล่าสุดที่เรา "แกล้งทำเป็นไม่รู้" (เช่น ปี 2024 ทั้งปี) เพื่อเอามาเทียบกับค่าที่โมเดลทาย









In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

 แบ่งข้อมูล `(ย้อนหลัง 12 เดือนสุดท้ายเป็น Test set)`

In [ ]:
train_data = gold_monthly[:-12]
test_data = gold_monthly[-12:]

 สร้างโมเดลจากข้อมูล Train` (ใช้ค่าที่หาได้จาก auto_arima)`

สมมติใช้ค่าที่ระบบหาให้ ถ้าเป็นค่าอื่นให้แก้ที่ order=(p, d, q)

In [ ]:
model_bt = ARIMA(train_data, order=auto_model.order)
model_bt_fit = model_bt.fit()

`ทำนายผล 12 เดือน`

In [ ]:
predictions = model_bt_fit.forecast(steps=12)
predictions.index = test_data.index

#  วัดผลความแม่นยำ(Measuring accuracy)

In [ ]:
mae = mean_absolute_error(test_data, predictions)
rmse = np.sqrt(mean_squared_error(test_data, predictions))

print(f"--- ผลการวัดความแม่นยำ ---")
print(f"ค่าเฉลี่ยความผิดพลาด (MAE): ${mae:.2f}")
print(f"Root Mean Squared Error (RMSE): ${rmse:.2f}")

In [ ]:
# 5. พล็อตกราฟเปรียบเทียบ
plt.figure(figsize=(12, 6))
plt.plot(train_data.tail(24), label='Training Data (Last 2 years)') # ดูย้อนหลัง 2 ปีพอให้เห็นภาพ
plt.plot(test_data, label='Actual Price', color='blue', marker='o')
plt.plot(predictions, label='Backtest Prediction', color='red', linestyle='--', marker='x')
plt.title('Gold Price Backtesting: Actual vs Predicted')
plt.legend()
plt.grid(True)
plt.show()

**Backtesting เอาไม่อยู่**


> นี่คือเหตุการณ์ปกติของโมเดล ARIMA เมื่อเจอกับสภาวะตลาดที่เปลี่ยนไปแบบฉับพลัน (Structural Break) หรือมีปัจจัยภายนอกที่ไม่เคยเกิดขึ้นมาก่อนในอดีตมาขับเคลื่อนราคาครับ






**วิเคราะห์จากกราฟ**


*  ` Underestimation: โมเดล ARIMA` พยายามพยากรณ์โดยอิงจากค่าเฉลี่ยและแนวโน้มเดิม ทำให้มันไม่สามารถจับ "การพุ่งขึ้นอย่างรุนแรง" (Bull Run) ของทองคำในปี 2024 ได้
*   `Lagging/Flat Effect`: เส้นสีแดงมีลักษณะเกือบแบนราบ แสดงว่าค่าความสัมพันธ์ในอดีต (p, d, q) ที่เราใช้อาจจะยังไม่พอที่จะรับมือกับความผันผวนระดับนี้ หรือข้อมูลรายเดือนอาจจะน้อยเกินไปสำหรับ ARIMA ในการเรียนรู้รอบวงจรรูปแบบใหม่



# AI (LSTM Model)


*   `Non-linear Relationships`: สามารถเรียนรู้ความสัมพันธ์ที่ซับซ้อนและไม่เป็นเส้นตรงได้ดีกว่า
*   `Multi-input`: เราสามารถใส่ปัจจัยอื่นนอกจากราคาทองเข้าไปได้ `(เช่น ดัชนีค่าเงินดอลลาร์)` เพื่อช่วยให้มันเข้าใจว่าทำไมราคาถึงพุ่ง
*   `Memory Gate`: มันสามารถเลือกว่าจะ "ลืม" ข้อมูลเก่าที่ใช้ไม่ได้แล้ว หรือ "จำ" รูปแบบการพุ่งของราคาในอดีตมาใช้ได้









In [ ]:
import sys
!{sys.executable} -m pip install tensorflow --only-binary :all:

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout